# **Competencia 2 - CC6205 Natural Language Processing 📚**

## Enunciado actualizado gracias a Ignacio Meza

Integrantes: Gerard Cathalifaud Salazar - Juan Pablo Herrera Pizarro

Usuario del equipo en CodaLab (Obligatorio):

Fecha límite de entrega 📆: 10 de Julio.

Tiempo estimado de dedicación:

Link competencia: Poner el link [aquí](https://codalab.lisn.upsaclay.fr/competitions/13646?secret_key=c2dbdef5-9869-4b0a-845a-2dc529b026fb)

### **Objetivo**

El objetivo de esta competencia es resolver una de las tareas más importantes en el área del procesamiento de lenguage natural, relacionada con la extracción de información: [Named Entity Recognition (NER)](http://www.cs.columbia.edu/~mcollins/cs4705-spring2019/slides/tagging.pdf).

En particular, y al igual que en la competencia anterior, deberán crear distintos modelos que apunten a resolver la tarea de NER en Español. Para esto, les entregaremos un dataset real perteneciente a la lista de espera NO GES en Chile. Es importante destacar que existe una falta de trabajos realizados en el área de NER en Español y aún más en el contexto clínico, por ende puede ser considerado como una tarea bien desafiante y quizás les interesa trabajar en el área más adelante en sus carreras.

En este notebook les entregaremos un baseline como referencia de los resultados que esperamos puedan obtener. Recuerden que el no superar a los baselines en alguna de las tres métricas conlleva un descuento de 0.5 puntos hasta 1.5 puntos.

Como hemos estado viendo redes neuronales tanto en catedras, tareas y auxiliares (o próximamente lo harán), esperamos que (por lo menos) utilicen Redes Neuronales Recurrentes (RNN) para resolverla.

Nuevamente, hay total libertad para utilizar el software y los modelos que deseen, siempre y cuando estos no traigan los modelos ya implementados. (De todas maneras como es un corpus nuevo, es difícil que haya algún modelo ya implementado con estas entidades)

### **Explicación de la competencia**

La tarea **NER** que van a resolver en esta competencia es comúnmente abordada como un problema de Sequence Labeling.

**¿Qué es Sequence Labeling?**

En breves palabras, dada una secuencia de tokens (oración) sequence labeling tiene por objetivo asignar una etiqueta a cada token de dicha secuencia. En pocas palabras, dada una lista de tokens esperamos encontrar la mejor secuencia de etiquetas asociadas a esa lista. Ahora veamos de qué se trata este problema.

**Named Entity Recognition (NER)**

NER es un ejemplo de un problema de Sequence Labeling. Pero antes de definir formalmente esta tarea, es necesario definir algunos conceptos claves para poder entenderla de la mejor manera:

- *Token*: Un token es una secuencia de caracteres, puede ser una palabra, un número o un símbolo.

- *Entidad*: No es más que un trozo de texto (uno o más tokens) asociado a una categoría predefinida. Originalmente se solían utilizar categorías como nombres de personas, organizaciones, ubicaciones, pero actualmente se ha extendido a diferentes dominios.

- *Límites de una entidad*: Son los índices de los tokens de inicio y fín dentro de una entidad.

- *Tipo de entidad*: Es la categoría predefinida asociada a la entidad.

Dicho esto, definimos formalmente una entidad como una tupla: $(s, e, t)$, donde $s, e$ son los límites de la entidad (índices de los tokens de inicio y fin, respectivamente) y t corresponde al tipo de entidad o categoría. Ya veremos más ejemplos luego de describir el Dataset.

**Corpus de la Lista de espera**

Trabajaran con un conjunto de datos reales correspondiente a interconsultas de la lista de espera NO GES en Chile. Si quieren saber más sobre cómo fueron generados los datos pueden revisar el paper publicado hace unos meses atrás en el workshop de EMNLP, una de las conferencias más importantes de NLP: [https://www.aclweb.org/anthology/2020.clinicalnlp-1.32/](https://www.aclweb.org/anthology/2020.clinicalnlp-1.32/).

Este corpus Chileno está constituido originalmente por 7 tipos de entidades pero por simplicidad en esta competencia trabajarán con las siguientes:

- **Disease**
- **Body_Part**
- **Medication**
- **Procedures**
- **Family_Member**

Si quieren obtener más información sobre estas entidades pueden consultar la [guía de anotación](https://plncmm.github.io/annodoc/). Además, mencionar que este corpus está restringido bajo una licencia que permite solamente su uso académico, así que no puede ser compartido más allá de este curso o sin permisos por parte de los autores en caso que quieran utilizarlo fuera. Si este último es el caso entonces pueden escribir directamente al correo: pln@cmm.uchile.cl. Al aceptar los términos y condiciones de la competencia están de acuerdo con los puntos descritos anteriormente.


**Formato ConLL**

Los archivos que serán entregados a ustedes vienen en un formato estándar utilizado en NER, llamado ConLL. No es más que un archivo de texto, que cumple las siguientes propiedades.

- Un salto de linea corresponde a la separación entre oraciones. Esto es importante ya que al entrenar una red neuronal ustedes pasaran una lista de oraciones como input, más conocidos como batches.

- La primera columna del archivo contiene todos los tokens de la partición.

- La segunda columna del archivo contiene el tipo de entidad asociado al token de la primera columna.

- Los tipos de entidades siguen un formato clásico en NER denominado *IOB2*. Si un tipo de entidad comienza con el prefijo "B-" (Beginning) significa que es el token de inicio de una entidad, si comienza con "I-" (Inside) es un token distinto al de inicio y si un token está asociado a la categoría O (Outside) significa que no pertenece a ninguna entidad.

Aquí va un ejemplo:

```
PACIENTE O
PRESENTA O
FRACTURA B-Disease
CORONARIA I-Disease
COMPLICADA I-Disease
EN O
PIE B-Body_Part
IZQUIERDO I-Body_Part
. O
SE O
REALIZA O
INSTRUMENTACION B-Procedure
INTRACONDUCTO I-Procedure
. O
```

Según nuestra definición tenemos las siguientes tres entidades (enumerando desde 0):

- $(2, 4, Disease)$
- $(6, 7, Body Part)$
- $(11, 12, Procedure)$

Repasen un par de veces todos estos conceptos antes de pasar a la siguiente sección del notebook.
Es importante entender bien este formato ya que al medir el rendimiento de sus modelos, consideraremos una **métrica estricta**. Esta métrica se llama así ya que considera correcta una predicción de su modelo, sólo si al compararlo con las entidades reales **coinciden tanto los límites de la entidad como el tipo.**

Para ejemplificar, tomando el caso anterior, si el modelo es capaz de encontrar la siguiente entidad: $(2, 3, Disease)$, entonces se considera incorrecto ya que pudo predecir dos de los tres tokens de dicha enfermedad. Es decir, buscamos una métrica que sea alta a nivel de entidad y no a nivel de token.

Antes de pasar a explicar las reglas, se recomienda visitar los siguientes links para entender bien el baseline de la competencia:

-  [Tagging, and Hidden Markov Models ](http://www.cs.columbia.edu/~mcollins/cs4705-spring2019/slides/tagging.pdf) (slides by Michael Collins), [notes](http://www.cs.columbia.edu/~mcollins/hmms-spring2013.pdf), [video 1](https://youtu.be/-ngfOZz8yK0), [video 2](https://youtu.be/PLoLKQwkONw), [video 3](https://youtu.be/aaa5Qoi8Vco), [video 4](https://youtu.be/4pKWIDkF_6Y)
-  [Recurrent Neural Networks](slides/NLP-RNN.pdf) | [video 1](https://youtu.be/BmhjUkzz3nk), [video 2](https://youtu.be/z43YFR1iIvk), [video 3](https://youtu.be/7L5JxQdwNJk)


Recuerden que todo el material se encuentra disponible en el [github del curso](https://github.com/dccuchile/CC6205).

### **Reglas de la competencia**

**texto en negrita**- Para que su competencia sea evaluada, deben participar en la competencia y enviar este notebook con su informe.
- Para participar, deben registrarse en la competencia en Codalab en grupos de máximo 4 alumnos. Cada grupo debe tener un nombre de equipo. (¡Y deben reportarlo en su informe, por favor!)
- Las métricas usadas serán métricas estrictas (ya explicado anteriormente) utilizando métricas clásicas como lo son precisión, recall y micro f1-score.
- En esta tarea se recomienda usar GPU. Pueden ejecutar su tarea en colab (lo cual trae todo instalado) o pueden intentar ejecutándolo en su computador. En este caso, deberá ser compatible con cuda y deberán instalar todo por su cuenta.
- En total pueden hacer un **máximo de 5 envíos**.
- Por favor, todas sus dudas haganlas por el canal de Discord. Los emails que lleguen al equipo docente serán remitidos a ese medio. Recuerden el ánimo colaborativo del curso.
- Estar top 5 en alguna de las tres métricas equivale a una bonificación en su nota final.

Éxito!


### **Baseline**

# En este punto esperamos que tengan conocimiento sobre redes neuronales y en particular redes neuronales recurrentes (RNN), si no siempre pueden escribirnos por el canal de Discord para aclarar dudas. La RNN del baseline adjunto a este notebook está programado en la librería [`pytorch`](https://pytorch.org/) pero ustedes pueden utilizar keras, tensorflow si así lo desean. El código contiene lo siguiente:

- La carga de los datasets, creación de batches de texto y padding (esto es importante ya que si utilizan redes neuronales tienen que tener el mismo largo los inputs).

- La implementación básica de una red `LSTM` simple de solo un nivel y sin bi-direccionalidad.

- La construcción del formato del output requerido para que lo puedan probar en la tarea en codalab.

Se espera que ustedes puedan experimentar con el baseline utilizando (pero no limitándose) estas sugerencias:

*   Probar la técnica de early stopping.
*   Variar la cantidad de parámetros de la capa de embeddings.
*   Variar la cantidad de capas RNN.
*   Variar la cantidad de parámetros de las capas de RNN.
*   Inicializar la capa de embeddings con modelos pre-entrenados. (word2vec, glove, conceptnet, etc...). [Embeddings en español aquí](https://github.com/dccuchile/spanish-word-embeddings). También aquí pueden encontrar unos embeddings clínicos en Español: [https://zenodo.org/record/3924799](https://zenodo.org/record/3924799)
*   Variar la cantidad de épocas de entrenamiento.
*   Variar el optimizador, learning rate, batch size, usar CRF loss, etc.
*   Probar una capa de CRF para garantizar el     formato IOB2.
*   Probar bi-direccionalidad.
*   Incluir dropout.
*   Probar modelos de tipo GRU.
*   Probar usando capas de atención.
*   Probar Embedding Contextuales (les puede ser de utilidad [flair](https://github.com/flairNLP/flair))
*   Probar modelos de transformers en español usando [Huggingface](https://github.com/huggingface/transformers) o el framework Flair.

### **Reporte**

Este debe cumplir la siguiente estructura:

1.	**Introducción**: Presentar brevemente el contexto, problema a resolver, incluyendo la formalización de la task (cómo son los inputs y outputs del problema) y los desafíos que ven al analizar el corpus entregado. (**0.5 puntos**)

2.	**Modelos**: Describir brevemente los modelos, métodos e hiperparámetros utilizados. (**1.0 puntos**)

4.	**Métricas de evaluación**: Describir las métricas utilizadas en la evaluación indicando qué miden y cuál es su interpretación en este problema en particular. (**0.5 puntos**)

5.  **Diseño experimental**: Esta es una de las secciones más importantes del reporte. Deben describir minuciosamente los experimentos que realizarán en la siguiente sección. Describir las variables de control que manejarán, algunos ejemplos pueden ser: Los hiperparámetros de los modelos, tipo de embeddings utilizados, tipos de arquitecturas. Ser claros con el conjunto de hiperparámetros que probarán, la decisión en las funciones de optimización, función de pérdida,  regulación, etc. Básicamente explicar qué es lo que veremos en la siguiente sección.
(**1 punto**)

6.	**Experimentos**: Reportar todos sus experimentos y código en esta sección. Comparar los resultados obtenidos utilizando diferentes modelos. ¡Es vital haber realizado varios experimentos para sacar una buena nota! (**2.0 puntos**)

7.	**Conclusiones**: Discutir resultados, proponer trabajo futuro. (**1 punto**)

# **Entregable.**

## **Introducción**


La lista de espera NO GES Chilena consiste en un listado de muchísimos casos clínicos registrados en los disintos locales médicos del país, informando detalles acerca del diagnóstico, tratamientos y otros datos adicionales. Al ser a escala nacional, se vuelve extremadamente dificil realizar una recopilación y análisis de la información, por lo que es necesario hacer algún trabajo para facilitar esta tarea.

Aquí nace la oportunidad de un ejercicio de Named Entity Recognition (NER), en la que consiste en rescatar y categorizar de las distintas palabras utilizadas en las oraciones, al tipo de entidad que pertenece, permitiendo así facilitar la búsqueda de estos, encontrar patrones, entre otras cosas.

El problema entonces consiste en generar una etiqueta para cada palabra, entre las opciones está que sea una parte del cuerpo, una enfermedad, un tratamiento médico, una medicación y un miembro familiar, adicionalmente de incluir una clasificación libre para las palabras que no quepan en ninguno de estas opciones.

Para realizar esto, se formatea la entrada de forma que se divide en dos columnas, la primera con las palabras y la segunda con la categoría a la que pertenece, donde el salto de línea simboliza en el término de una oración. Además, las entidades vienen con un prefijo que busca señalar cuando es el comienzo de la entidad (B-) o es una palabra intermedia o final (I-).

Para la salida, se entrega una tupla de 3 elementos, los primeros dos para indicar el inicio y fin de la entidad, y la tercera para indicar dicha entidad.

El problema principal de esta tarea consiste en el vocabulario especializado, donde se debe utilizar el contexto para determinar de mejor manera a que entidad pertecene y si está es parte o no de una entidad compuesta por más de 1 palabra. Además, no existe mucho trabajo sobre esta área en el idioma español.

## **Modelos**


Entre los métodos, se escogió utilizar la un embedding ya entrenado anteriormente para una tarea similar, es decir, se utilizó un vocabulario en donde ya los vectores fueron codificados de forma que tengan una relación contextual/semántica efectiva por las palabras representadas bajo el pretexto de una situación médica.

## **Métricas de evaluación**



- **Métrica estricta:**
La metríca estricta consiste en que además de evaluar la correcta categorización de cada token por la entidad correspondiente, debe efectivamente reconocerle todos los tokens a la misma entidad, es decir debe contar con todos los tokens en lo que forma parte y empezar donde corresponda, sino se considerará como una categorización errónea.
- **Precision:**
Esta medida, trata de la razón entre las clasificaciones correctas de la entidad con respecto al total de veces indicado que los tokens pertenecen a dicha entidad (que incluye las incorrectamente indicadas, es decir, falso positivo). En resumen, indica que tan "preciso" fue al momento de indicar que el token pertenecía a una entidad sin haberse equivocado.
- **Recall:**
Esta otra medida, corresponde al número de correctas clasificaciones a una entidad sobre al número deal de tokens que corresponden a dicha entidad (Que incluye las que se categorizó como no pertenecientes, es decir, falsos negativo). Resumiendo, indica si no dejo fuera de la entidad a los tokens que pertenecían a ella.
- **Micro/Macro F1 score:**
F1 score, consiste en un resumen de las medidas Precision y Recall, es su promedio armónico, luego para obtener el puntaje general, se determina el promedio de F1 score de todas las entidades, sin embargo, existen distintas formas de obtener este promedio, la diferencia entre ser un micro o macro radica en la forma de como se obtiene dicho promedio.
Macro es simplemente el promedio aritmético de los F1 scores de cada entidad, es decir, se calcula F1 score por cada entidad y luego se promedian. En cambio, la F1 micro trata de F1 score general, donde se calcula según el total de positivos, dividido por la suma del total de positivos y la mitad de los falsos positivos con los falsos negativos.

## **Diseño experimental**

Para la primera parte se decidió utilizar el embedding ya entrenado, disponible  [aquí](https://zenodo.org/record/3924799/), consiste en un enorme vocabulario utilizado en el contexto de lista de espera chilena, por lo tanto, perteneciente a un conjunto mayor que contiene al de la lista de espera no ges chilena. Lo cual facilita el proceso de aprendizaje al tener ya ciertas relaciones venidas del embedding.

Para esto, al momentro de construir el vocabulario, se utilizó el de este embedding más el del corpus, que iniciliazó de manera aleatoria, dejando libre la modificación de estos vectores según el entrenamiento de la red.

## **Experimentos**


El código que les entregaremos servirá de baseline para luego implementar mejores modelos.
En general, el código asociado a la carga de los datos, las funciones de entrenamiento, de evaluación y la predicción de los datos de la competencia no deberían cambiar.
Solo deben preocuparse de cambiar la arquitectura del modelo, sus hiperparámetros y reportar, lo cual lo pueden hacer en las subsecciones *modelos*.



###  **Carga de datos y Preprocesamiento**

Para cargar los datos y preprocesarlos usaremos la librería [`torchtext`](https://github.com/pytorch/text). Tener cuidado ya que hace algunos meses esta librería tuvo cambios radicales, quedando las funcionalidades pasadas depreciadas de la librería ```legacy```. Esto ya que si quieren usar más funciones de la librería entonces vean los cambios en la documentación debe usar la versión antigua con python 3.8




El proceso será el siguiente:

1. Descargar los datos desde github y examinarlos.
2. Cargar los datasets con la clase ```TaggingDataset``` de más abajo.
3. Crear el vocabulario.

In [ ]:
!pip install -U torchtext

In [40]:
import torch
import torchtext
from torchtext import data, datasets

# Garantizar reproducibilidad de los experimentos
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

---
#### Embedding Médico
---

In [41]:
!wget https://zenodo.org/record/3924799/files/cwlce.vec?download=1 -nc -O cwlce.vec

File ‘cwlce.vec’ already there; not retrieving.


In [42]:
import gensim
from gensim.models import KeyedVectors

medicalW2V = KeyedVectors.load_word2vec_format("cwlce.vec", binary=False) # Utilizando el embedding médico.
#medicalW2V_weights = torch.FloatTensor(medicalW2V.vectors) # Transformando en un tensor float.

In [ ]:
print(medicalW2V.key_to_index["dolor"])
print(medicalW2V.get_vector("dolor"))
#medicalW2V_weights

35
[ 1.33799e-01  9.61610e-02 -1.30059e-01 -4.43090e-01  4.47357e-01
 -1.05935e-01 -1.03139e-01  2.19270e-02 -2.26524e-01  1.75927e-01
 -1.52117e-01  3.75409e-01  5.87510e-02 -9.27570e-02 -1.67034e-01
 -2.29453e-01  4.26858e-01 -6.62080e-02  2.09038e-01  7.59360e-02
 -2.95280e-02  2.29779e-01  2.67009e-01 -6.46560e-02  1.55142e-01
  8.39750e-02  6.23110e-02  4.93711e-01  3.26826e-01  4.36605e-01
 -1.22404e-01  3.43340e-02 -1.05549e-01 -1.71808e-01  1.19521e-01
 -2.00356e-01 -6.02500e-02 -3.60321e-01  2.40590e-01 -1.44226e-01
  1.29498e-01 -1.28684e-01 -1.26623e-01 -2.77384e-01 -1.84895e-01
  1.31613e-01 -1.60732e-01 -2.96633e-01  2.47983e-01  2.39840e-02
  7.18140e-02  8.75290e-02  1.57100e-02 -8.91330e-02 -1.12503e-01
 -1.45488e-01  2.36185e-01 -9.94790e-02 -7.85920e-02  2.12178e-01
  2.04644e-01  3.26968e-01 -4.34120e-02 -2.73758e-01 -1.30110e-02
  2.13482e-01 -5.88710e-02 -1.06041e-01 -1.09029e-01  3.65386e-01
  3.43756e-01 -1.03250e-02  2.39566e-01 -3.15295e-01 -2.43540e-02
 -1.930

#### **Obtener datos**

Descargamos los datos de entrenamiento, validación y prueba en nuestro directorio de trabajo

In [43]:
#%%capture

!wget https://github.com/dccuchile/CC6205/releases/download/v1.0/train.txt -nc # Dataset de Entrenamiento
!wget https://github.com/dccuchile/CC6205/releases/download/v1.0/dev.txt -nc    # Dataset de Validación (Para probar y ajustar el modelo)
!wget https://github.com/dccuchile/CC6205/releases/download/v1.0/test.txt -nc  # Dataset de la Competencia. Estos datos solo contienen los tokens. ¡¡SON LOS QUE DEBEN SER PREDICHOS!!

File ‘train.txt’ already there; not retrieving.

File ‘dev.txt’ already there; not retrieving.

File ‘test.txt’ already there; not retrieving.



In [44]:
# NUEVO DATALOADER Y OTRAS COSAS NECESARIAS
from collections import Counter, OrderedDict

import torch

from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from torchtext.vocab import vocab

class TaggingDataset(Dataset):
    def __init__(self, path, lower=False, separator=" ", encoding="utf-8"):

        with open(path, 'r', encoding=encoding) as file:
          text, tag, data = [], [], []
          for line in file:
              line = line.strip()
              if line == "":
                  data.append(dict({'text':text, 'nertags':tag}))
                  text, tag = [], []
              else:
                  line_content = line.split(separator) # .rstrip('\n')
                  if lower:
                    text.append(line_content[0].lower())
                  else:
                    text.append(line_content[0])
                  tag.append(line_content[1])
        data.append(dict({'text':text, 'nertags':tag}))

        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        text = item["text"]
        nertags = item["nertags"]
        return nertags, text

def fit_vocab(data_iter):

  def update_counter(counter_obj):
    sorted_by_freq_tuples = sorted(counter_obj.items(),
                                  key=lambda x: x[1],
                                  reverse=True)
    ordered_dict = OrderedDict(sorted_by_freq_tuples)
    return ordered_dict

  counter_1 = Counter()
  counter_2 = Counter()
  for _nertags, _text in data_iter:
    counter_1.update(_text)
    counter_2.update(_nertags)

  od1 = update_counter(counter_1)
  od2 = update_counter(counter_2)

  v1 = vocab(od1, specials=['<PAD>', '<unk>'])
  v1.set_default_index(v1["<unk>"])
  v2 = vocab(od2, specials=['<PAD>'])

  text_pipeline = lambda x: v1(x)
  nertags_pipeline = lambda x: v2(x)

  return text_pipeline, nertags_pipeline, v1, v2

def collate_batch(batch):
  nertags_list, text_list = [], []
  for _nertags, _text in batch:
    processed_nertags = torch.tensor(nertags_pipeline(_nertags),
                                     dtype=torch.int64)
    nertags_list.append(processed_nertags)
    processed_text = torch.tensor(text_pipeline(_text),
                                  dtype=torch.int64)
    text_list.append(processed_text)
  nertags_list = pad_sequence(nertags_list, batch_first=True).T
  text_list = pad_sequence(text_list, batch_first=True).T
  return nertags_list.to(device), text_list.to(device)

In [45]:
def fit_vocab_preEmbedding(data_iter, Embedding_pretrained):

  def update_counter(counter_obj):
    sorted_by_freq_tuples = sorted(counter_obj.items(),
                                  key=lambda x: x[1],
                                  reverse=True)
    ordered_dict = OrderedDict(sorted_by_freq_tuples)
    return ordered_dict

  counter_1 = Counter()
  counter_2 = Counter()

  list_medic = []
  for word in Embedding_pretrained.index_to_key:
    list_medic.append((word, 1))
  od1 = OrderedDict(list_medic)

  for _nertags, _text in data_iter:
    if not _text in list_medic:
      counter_1.update(_text)
    counter_2.update(_nertags)

  od0 = update_counter(counter_1)
  od2 = update_counter(counter_2)

  for word in od0:
    od1[word] = 1

  v1 = vocab(od1, specials=['<PAD>', '<unk>'])
  v1.set_default_index(v1["<unk>"])
  v2 = vocab(od2, specials=['<PAD>'])
  # vocab_pretrained_embedding = Embedding_pretrained.index_to_key
  # for index in range(len(vocab_pretrained_embedding)):
  #   v1.vocab.insert_token(vocab_pretrained_embedding[index], length_v1_corpus + index)

  text_pipeline = lambda x: v1(x)
  nertags_pipeline = lambda x: v2(x)

  return text_pipeline, nertags_pipeline, v1, v2

In [46]:
train_iter = TaggingDataset("train.txt")
dev_iter = TaggingDataset("dev.txt")
test_iter = TaggingDataset("test.txt")

text_pipeline, nertags_pipeline, TEXT, NER_TAGS = fit_vocab(train_iter)

In [47]:
train_iter_lower = TaggingDataset("train.txt", lower = True)
dev_iter_lower = TaggingDataset("dev.txt", lower = True)
test_iter_lower = TaggingDataset("test.txt", lower = True)

text_pipeline, nertags_pipeline, TEXT, NER_TAGS = fit_vocab_preEmbedding(train_iter_lower, medicalW2V)

In [48]:
# seteamos algunos valores de interes
UNK_IDX = TEXT.vocab.get_stoi()['<unk>']
PAD_IDX = TEXT.vocab.get_stoi()['<PAD>']

PAD_TAG_IDX = NER_TAGS.get_stoi()['<PAD>']
O_TAG_IDX = NER_TAGS.vocab.get_stoi()['O']

In [24]:
BATCH_SIZE = 22

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

dataloader_train = DataLoader(
    train_iter, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch
)
dataloader_dev = DataLoader(
    dev_iter, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch
)
dataloader_test = DataLoader(
    test_iter, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch
)

Using cuda


In [49]:
BATCH_SIZE = 22

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

dataloader_train = DataLoader(
    train_iter_lower, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch
)
dataloader_dev = DataLoader(
    dev_iter_lower, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch
)
dataloader_test = DataLoader(
    test_iter_lower, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch
)

Using cuda


In [50]:
example = next(iter(dataloader_train))
text_example = example[1]

print (len(text_example[:, 0]))

# revisamos el primer ejemplo
[TEXT.vocab.get_itos()[j] for j in text_example[:, 0]]

42


['k08',
 '.',
 '1',
 '-',
 'perdida',
 'de',
 'dientes',
 'debida',
 'a',
 'accidente',
 ',',
 'extraccion',
 'o',
 'enf',
 '.',
 'periodontal',
 'local',
 '/',
 'se',
 'solicita',
 'protesis',
 'parcial',
 'superior',
 'e',
 'inferior',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>']

In [51]:
!pip install seqeval

In [52]:
# Definimos las métricas

from seqeval.metrics import f1_score, precision_score, recall_score

def calculate_metrics(preds, y_true, pad_idx=PAD_TAG_IDX, o_idx=O_TAG_IDX):
    """
    Calcula precision, recall y f1 de cada batch.
    """

    # Obtener el indice de la clase con probabilidad mayor. (clases)
    y_pred = preds.argmax(dim=1, keepdim=True)

    # filtramos <pad> para calcular los scores.
    mask = [(y_true != pad_idx)]
    y_pred = y_pred[mask]
    y_true = y_true[mask]

    # traemos a la cpu
    y_pred = y_pred.view(-1).to('cpu').numpy()
    y_true = y_true.to('cpu').numpy()
    y_pred = [[NER_TAGS.vocab.get_itos()[v] for v in y_pred]]
    y_true = [[NER_TAGS.vocab.get_itos()[v] for v in y_true]]

    # calcular scores
    f1 = f1_score(y_true, y_pred, mode='strict')
    precision = precision_score(y_true, y_pred, mode='strict')
    recall = recall_score(y_true, y_pred, mode='strict')

    return precision, recall, f1

-------------------

### **Modelo Baseline**

Teniendo ya cargado los datos, toca definir nuestro modelo. Este baseline tendrá una capa de embedding, unas cuantas LSTM y una capa de salida y usará dropout en el entrenamiento.

Este constará de los siguientes pasos:

1. Definir la clase que contendrá la red.
2. Definir los hiperparámetros e inicializar la red.
3. Definir el número de épocas de entrenamiento
4. Definir la función de loss.



Recomendamos que para experimentar, encapsules los modelos en una sola variable y luego la fijes en model para entrenarla

In [21]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# Definir la red
class NER_RNN(nn.Module):
    def __init__(self,
                 input_dim,
                 embedding_dim,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout,
                 pad_idx):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding(input_dim,
                                      embedding_dim,
                                      padding_idx=pad_idx,
                                      )

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))

        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

#### **Hiperparámetros de la red**



In [22]:
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.6
BIDIRECTIONAL = True

# Creamos nuestro modelo.
baseline_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

baseline_model_name = 'baseline'  # nombre que tendrá el modelo guardado...

In [23]:
baseline_n_epochs = 20

#### Definimos la función de loss

In [26]:
# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.get_stoi()['<PAD>']
baseline_criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

# Designando modelo test
baseline_test = True
model1_test = False
model2_test = False
model3_test = False

--------------------
### Modelo 1

En estas secciones pueden implementar nuevas redes al modificar los hiperparámetros, la cantidad de épocas de entrenamiento, el tamaño de los batches, loss, optimizador, etc... como también definir nuevas arquitecturas de red (mediante la creación de clases nuevas)


Al final de estas, hay 4 variables, las cuales deben setear con los modelos, épocas de entrenamiento, loss y optimizador que deseen probar.


In [53]:
import numpy
len(torch.cat((torch.tensor([numpy.zeros(300, dtype=numpy.double), numpy.zeros(300, dtype=numpy.double)]), torch.DoubleTensor(medicalW2V.vectors))))

<ipython-input-53-e89e86985425>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  len(torch.cat((torch.tensor([numpy.zeros(300, dtype=numpy.double), numpy.zeros(300, dtype=numpy.double)]), torch.DoubleTensor(medicalW2V.vectors))))


57114

In [55]:
# model_1 = ...
# model_name_1 = ...
# n_epochs_1 = ...
# loss_1 = ...

import numpy

# Definir la red
class Model1(nn.Module):
    def __init__(self,
                 input_dim,
                 pretained_Embedding,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout,
                 pad_idx):

        super().__init__()

        # Capa de embedding
        # Dejamos como un embedding pre-entrenado que se puede volver a modificar durante el entrenamiento de la red.
        embedding_dim = 300
        pretained_Embedding_weights = torch.cat((torch.tensor([numpy.zeros(embedding_dim, dtype=numpy.single), numpy.zeros(embedding_dim, dtype=numpy.single)], dtype = torch.float), torch.FloatTensor(medicalW2V.vectors))) # Transformando en un tensor float.

        pretained_Embedding_weights = torch.cat((pretained_Embedding_weights, torch.rand(input_dim - len(pretained_Embedding_weights), embedding_dim, dtype = torch.float)))


        self.embedding = torch.nn.Embedding.from_pretrained(pretained_Embedding_weights, freeze = False, padding_idx = pad_idx)


        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))

        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
pretained_Embedding = medicalW2V
input_dim = len(TEXT.vocab)
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.6
BIDIRECTIONAL = True

# Creamos nuestro modelo.
model_1 = Model1(input_dim, pretained_Embedding, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

model_1_name = 'baseline_medicEmbedding'  # nombre que tendrá el modelo guardado...
model_1_n_epochs = 20
# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.get_stoi()['<PAD>']
model_1_criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

# Designando modelo test
baseline_test = False
model1_test = True
model2_test = False
model3_test = False

---------------

### Modelo 2

In [ ]:
# model_2 = ...
# model_name_2 = ...
# n_epochs_2 = ...
# loss_2 = ...

---------------


### Modelo 3

In [ ]:
# modelo_3 = ...
# model_name_3 = ...
# n_epochs_3 = ...
# loss_3 = ...

------
### **Entrenamos y evaluamos**


**Importante** : Fijen el modelo, el número de épocas de entrenamiento, la loss y el optimizador que usarán para entrenar y evaluar en las siguientes variables!!!

In [57]:
if baseline_test:
  model = baseline_model
  model_name = baseline_model_name
  criterion = baseline_criterion
  n_epochs = baseline_n_epochs
  print("model test: Baseline")

if model1_test:
  model = model_1
  model_name = model_1_name
  criterion = model_1_criterion
  n_epochs = model_1_n_epochs
  print("model test: Model 1")

model test: Model 1




#### **Inicializamos la red**

Iniciamos los pesos de la red de forma aleatoria (Usando una distribución normal).


In [28]:
def init_weights(m):
    # Inicializamos los pesos como aleatorios
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1)

    # Seteamos como 0 los embeddings de UNK y PAD.
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

model.apply(init_weights)

NER_RNN(
  (embedding): Embedding(17591, 300, padding_idx=0)
  (lstm): LSTM(300, 256, num_layers=3, dropout=0.6, bidirectional=True)
  (fc): Linear(in_features=512, out_features=12, bias=True)
  (dropout): Dropout(p=0.6, inplace=False)
)

In [58]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

El modelo actual tiene 22,510,160 parámetros entrenables.


Notar que definimos los embeddings que representan a \<unk\> y \<pad\>  como [0, 0, ..., 0]

#### **Definimos el optimizador**

In [59]:
# Optimizador
optimizer = optim.Adam(model.parameters())

#### **Enviamos el modelo a cuda**


In [60]:
# Enviamos el modelo y la loss a cuda (en el caso en que esté disponible)
model = model.to(device)
criterion = criterion.to(device)

#### **Definimos el entrenamiento de la red**

Algunos conceptos previos:

- `epoch` : una pasada de entrenamiento completa de una dataset.
- `batch`: una fracción de la época. Se utilizan para entrenar mas rápidamente la red. (mas eficiente pasar n datos que uno en cada ejecución del backpropagation)

Esta función está encargada de entrenar la red en una época. Para esto, por cada batch de la época actual, predice los tags del texto, calcula su loss y luego hace backpropagation para actualizar los pesos de la red.

Observación: En algunos comentarios aparecerá el tamaño de los tensores entre corchetes

In [61]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.train()

    # Por cada batch del iterador de la época:
    for tags, text in iterator:
        # Reiniciamos los gradientes calculados en la iteración anterior
        optimizer.zero_grad()

        #text = [sent len, batch size]

        # Predecimos los tags del texto del batch.
        predictions = model(text.to(device))

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]

        # Reordenamos los datos para calcular la loss
        predictions = predictions.view(-1, predictions.shape[-1])
        #ipdb.set_trace()
        tags = torch.reshape(tags, (-1,)).to(device)

        #predictions = [sent len * batch size, output dim]



        # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
        loss = criterion(predictions, tags)

        # Calculamos el accuracy
        precision, recall, f1 = calculate_metrics(predictions, tags)

        # Calculamos los gradientes
        loss.backward()

        # Actualizamos los parámetros de la red
        optimizer.step()

        # Actualizamos el loss y las métricas
        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

#### **Definimos la función de evaluación**

Evalua el rendimiento actual de la red usando los datos de validación.

Por cada batch de estos datos, calcula y reporta el loss y las métricas asociadas al conjunto de validación.
Ya que las métricas son calculadas por cada batch, estas son retornadas promediadas por el número de batches entregados. (ver linea del return)

In [62]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()

    # Indicamos que ahora no guardaremos los gradientes
    with torch.no_grad():
        # Por cada batch
        for tags, text in iterator:
            # Predecimos
            predictions = model(text.to(device))

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = torch.reshape(tags, (-1,)).to(device)

            # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
            loss = criterion(predictions, tags)

            # Calculamos las métricas
            precision, recall, f1 = calculate_metrics(predictions, tags)

            # Actualizamos el loss y las métricas
            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

In [63]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


#### **Entrenamiento de la red**

En este cuadro de código ejecutaremos el entrenamiento de la red.
Para esto, primero definiremos el número de épocas y luego por cada época, ejecutaremos `train` y `evaluate`.

**Importante: Reiniciar los pesos del modelo**

Si ejecutas nuevamente esta celda, se seguira entrenando el mismo modelo una y otra vez.
Para reiniciar el modelo se debe ejecutar nuevamente la celda que contiene la función `init_weights`



In [64]:
best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.time()

    # Recuerdo: dataloader_train y valid_iterator contienen el dataset dividido en batches.

    # Entrenar
    train_loss, train_precision, train_recall, train_f1 = train(
        model, dataloader_train, optimizer, criterion)

    # Evaluar (valid = validación)
    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
        model, dataloader_dev, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # Si obtuvimos mejores resultados, guardamos este modelo en el almacenamiento (para poder cargarlo luego)
    # Si detienen el entrenamiento prematuramente, pueden cargar el modelo en el siguiente recuadro de código.
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
    # Si ya no mejoramos el loss de validación, terminamos de entrenar.

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(
        f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
    )
    print(
        f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
    )

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <PAD> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 13s
	Train Loss: 0.696 | Train f1: 0.48 | Train precision: 0.59 | Train recall: 0.42
	 Val. Loss: 0.367 |  Val. f1: 0.74 |  Val. precision: 0.79 | Val. recall: 0.71
Epoch: 02 | Epoch Time: 0m 12s
	Train Loss: 0.377 | Train f1: 0.73 | Train precision: 0.76 | Train recall: 0.70
	 Val. Loss: 0.295 |  Val. f1: 0.78 |  Val. precision: 0.79 | Val. recall: 0.78
Epoch: 03 | Epoch Time: 0m 11s
	Train Loss: 0.302 | Train f1: 0.78 | Train precision: 0.80 | Train recall: 0.76
	 Val. Loss: 0.270 |  Val. f1: 0.80 |  Val. precision: 0.81 | Val. recall: 0.79
Epoch: 04 | Epoch Time: 0m 11s
	Train Loss: 0.257 | Train f1: 0.81 | Train precision: 0.82 | Train recall: 0.80
	 Val. Loss: 0.261 |  Val. f1: 0.82 |  Val. precision: 0.81 | Val. recall: 0.82
Epoch: 05 | Epoch Time: 0m 11s
	Train Loss: 0.226 | Train f1: 0.83 | Train precision: 0.84 | Train recall: 0.83
	 Val. Loss: 0.266 |  Val. f1: 0.81 |  Val. precision: 0.81 | Val. recall: 0.81
Epoch: 06 | Epoch Time: 0m 12s
	Train Lo

**Importante**: Recuerden que el último modelo entrenado no es el mejor (probablemente esté *overfitteado*), si no el que guardamos con la menor loss del conjunto de validación. Este problema lo pueden solucionar con *early stopping*.
Para cargar el mejor modelo entrenado, ejecuten la siguiente celda.



In [65]:
# cargar el mejor modelo entrenado.
model.load_state_dict(torch.load('{}.pt'.format(model_name)))

<All keys matched successfully>

In [66]:
# Limpiar ram de cuda
torch.cuda.empty_cache()

In [67]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
    model, dataloader_dev, criterion)

print(
    f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
)

Val. Loss: 0.261 |  Val. f1: 0.82 | Val. precision: 0.81 | Val. recall: 0.82


#### **Evaluamos el set de validación con el modelo final**

Estos son los resultados de predecir el dataset de evaluación con el *mejor* modelo entrenado.

In [68]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
    model, dataloader_dev, criterion)

print(
    f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
)

Val. Loss: 0.261 |  Val. f1: 0.82 | Val. precision: 0.81 | Val. recall: 0.82


### **Predecir datos para la competencia**

Ahora, a partir de los datos de **test** y nuestro modelo entrenado, vamos a predecir las etiquetas que serán evaluadas en la competencia.

In [ ]:
def predict_labels(model, iterator, criterion, fields=(TEXT, NER_TAGS)):

    # Extraemos los vocabularios.
    text_field = fields[0]
    nertags_field = fields[1]
    tags_vocab = nertags_field.vocab.get_itos()
    words_vocab = text_field.vocab.get_itos()

    model.eval()

    predictions = []

    with torch.no_grad():

        for tags, text in iterator:

            text_batch = text
            text_batch = torch.transpose(text_batch, 0, 1).tolist()

            # Predecir los tags de las sentences del batch
            predictions_batch = model(text)
            predictions_batch = torch.transpose(predictions_batch, 0, 1)

            # por cada oración predicha:
            for sentence, sentence_prediction in zip(text_batch,
                                                     predictions_batch):
                for word_idx, word_predictions in zip(sentence,
                                                      sentence_prediction):
                    # Obtener el indice del tag con la probabilidad mas alta.
                    argmax_index = word_predictions.topk(1)[1]

                    current_tag = tags_vocab[argmax_index]
                    # Obtenemos la palabra
                    current_word = words_vocab[word_idx]

                    if current_word != '<PAD>':
                        predictions.append([current_word, current_tag])
                predictions.append(['EOS', 'EOS'])


    return predictions


predictions = predict_labels(model, dataloader_test, criterion)

### **Generar el archivo para la submission**

No hay problema si aparecen unk en la salida. Estos no son relevantes para evaluarlos, usamos solo los tags.

In [ ]:
import os, shutil

if (os.path.isfile('./predictions.zip')):
    os.remove('./predictions.zip')

if (not os.path.isdir('./predictions')):
    os.mkdir('./predictions')

else:
    # Eliminar predicciones anteriores:
    shutil.rmtree('./predictions')
    os.mkdir('./predictions')

f = open('predictions/predictions.txt', 'w')
for i, (word, tag) in enumerate(predictions[:-1]):
    if word=='EOS' and tag=='EOS': f.write('\n')
    else:
      if i == len(predictions[:-1])-1:
        f.write(word + ' ' + tag)
      else: f.write(word + ' ' + tag + '\n')

f.close()

a = shutil.make_archive('predictions', 'zip', './predictions')

## **Conclusiones**



    Escriba aquí sus conclusiones